# AutoPrognosis API Tutorial

A demonstration for AP functionality and operation

This tutorial shows how to use [Autoprognosis](https://arxiv.org/abs/1802.07207). 

See [installation instructions](../../doc/install.md) to install the dependencies.

In [2]:
import pandas as pd
import numpy as np
import initpath_ap
initpath_ap.init_sys_path()
import utilmlab
import json;
from scipy import stats
#import AutoPrognosis Library:
import model

# Run the model from command line

## introduce with a sample

In [11]:
df = pd.read_csv('../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_vascular.csv')
df

,eid,gender,age-0,average-sys-0,history-of-diabetes,hypertention-medication-0,smoker,average-BMI-0,prob,outcome
0,4049399,0,40.8,128.0,False,False,0.0,25.9,0.076822,False
1,2461215,1,54.0,132.5,False,False,0.0,24.7,0.570690,False
2,1829764,1,59.6,137.5,False,False,0.0,27.8,0.274643,False
3,5592240,0,46.1,122.5,False,False,0.0,20.0,0.040465,False
4,5725915,0,59.1,147.5,False,False,0.0,36.8,0.066206,False
...,...,...,...,...,...,...,...,...,...,...
22739,5614020,1,44.5,147.5,False,False,0.0,26.5,0.005060,False
22740,1979261,0,58.7,130.5,False,False,0.0,23.9,0.001221,False
22741,5859580,1,40.4,136.5,False,False,0.0,26.8,0.001032,False
22742,3038532,1,48.8,164.0,False,False,1.0,30.7,0.365570,False


-i : the input csv file

--target : the name of the column that contains the outcome (what you want to predict for the validation/test set)

-o : the folder in which the output of AutoPrognosis is written

--it : total number of iterations for each fold or n-fold cross validation

--cv : If 0, that means a normal validation with train and test (or validation) set. -iValIndex should also be set. Otherwise, n for n-fold cross validation

-iValIndex: address of the test index file, test_indexes.csv or val_indexes.csv", 

--nstage: size of pipeline: 0: auto (selects imputation when missing data is detected),
        1: only classifiers, 
        2: feature processesing + clf, 
        3: imputers + feature processors and clf
        4: imputers (if needed) + clf
        
--ensemble : include ensembles when fitting. It gives an assertion error when set to 0! should be looked into.

--modelindexes : list of classifiers that we want to try

0 Random Forest,
1 Gradient Boosting, 
2 XGBoost, 
3 Adaboost, 
4 Bagging, 
5 Bernoulli Naive Bayes, 
6 Gauss Naive Bayes, 
7 Multinomial Naive Bayes, 
8 Logistic Regression, 
9 Perceptron, 
10 Decision Trees, 
11 QDA, 
12 LDA, 
13 KNN, 
14 Linear SVM, 
15 Neural Network

### Use AutoPrognosis with cross validation

In [12]:
!python3 autoprognosis.py\
-i ../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_vascular.csv\
--target outcome \
-o ../../../AutoPrognosisThings/outputs \
--it 15 \
--cv 3 \
--nstage 4 \
--modelindexes 2\
--num_components 1\
--kernel_freq 100

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 5s (5s) (70s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 6s (3s) (42s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.0
[ XGBoost ]
Iteration number: 3 7s (2s) (33s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3424127458434802
[ XGBoost ]
Iteration number: 4 7s (2s) (28s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2871349032045143
[ XGBoost ]
Iteration number: 5 9s (2s) (26s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6903393940181122
[ XGBoost ]
Iteration number: 6 10s (2s) (24s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.7776979952036847
[ XGBoost ]
Iteration number: 7 11s (2s) (23s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.5305158337751459
[ XGBoost ]
Iteration number: 8 12s (1s) (22s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.6041377479673005
[ XGBoost ]
Iteration number: 9 12s (1s) (21s), Current pipelines:  [[[ 

In [13]:
!python3 autoprognosis_report.py -i ../../../AutoPrognosisThings/outputs --verbose 1

Score

classifier      aucroc 0.609
classifier      aucprc 0.010
ensemble        aucroc 0.623
ensemble        aucprc 0.011

Report

best score single pipeline (while fitting)    0.558
model_names_single_pipeline                   [ XGBoost ]
best ensemble score (while fittng)            0.604
ensemble_pipelines                            ['[ XGBoost ]', '[ XGBoost ]', '[ XGBoost ]']
ensemble_pipelines_weight                     [0.0, 0.07197840433455004, 0.92802159566545]
optimisation_metric                           aucroc
hyperparameter_properties                     [{'name': 'XGBoost', 'hyperparameters': {'model': "XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,\n              colsample_bynode=1, colsample_bytree=0.6037599731551904,\n              gamma=5.998657591844557, gpu_id=-1, importance_type='gain',\n              interaction_constraints='', learning_rate=0.11772659593910406,\n              max_delta_step=0, max_depth=4,\n              min_child_weight=0

### Use AutoPrognosis with train and validation set

In [14]:
!python3 autoprognosis.py\
-i ../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_vascular.csv\
-iValIndex ../../../AutoPrognosisThings/cardio_data/val_indexes.csv\
--target outcome \
-o ../../../AutoPrognosisThings/outputs \
--it 15 \
--cv 0 \
--nstage 4 \
--modelindexes 2\
--num_components 1\
--kernel_freq 100

[ XGBoost ]
[ XGBoost ]
Iteration number: 1 1s (1s) (8s), Current pipelines:  [[[ XGBoost ]]], BO objective: 0.0
[ XGBoost ]
Iteration number: 2 1s (0s) (7s), Current pipelines:  [[[ XGBoost ]]], BO objective: -0.9999999999999971
[ XGBoost ]
Iteration number: 3 1s (0s) (7s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1436919220164126
[ XGBoost ]
Iteration number: 4 2s (0s) (7s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1629302500688166
[ XGBoost ]
Iteration number: 5 2s (0s) (7s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.1778641023427532
[ XGBoost ]
Iteration number: 6 3s (0s) (7s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.2346151646182941
[ XGBoost ]
Iteration number: 7 3s (0s) (7s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.3617074037630583
[ XGBoost ]
Iteration number: 8 5s (1s) (10s), Current pipelines:  [[[ XGBoost ]]], BO objective: -1.181594687619073
[ XGBoost ]
Iteration number: 9 6s (1s) (10s), Current pipelines:  [

# Run the model by short simple python code

In [16]:
df_all= pd.read_csv('../../../AutoPrognosisThings/cardio_data/withImage2/with_prob_no_test_vascular.csv')
X_= df_all.drop(columns=['outcome'])
Y_= df_all[['outcome']]

In [24]:
metric = 'aucprc'
acquisition_type = 'MPI' # default and prefered is LCB but this generates excessive warnings, MPI is a good compromise.
model.nmax_model= 4 #this is the same as nstage 
AP_mdl   = model.AutoPrognosis_Classifier(
    metric=metric, CV=3, num_iter=3, kernel_freq=100, ensemble=True,
    ensemble_size=3, Gibbs_iter=100, burn_in=50, num_components=1, 
    acquisition_type=acquisition_type, my_model_indexes=[2])

In [25]:
AP_mdl.fit(X_, Y_)

[ iterative_extra_trees, XGBoost ]


[ median, XGBoost ]


Iteration number: 1 2s (2s) (26s), Current pipelines:  [[[ iterative_k_neighbors, XGBoost ]]], BO objective: 0.0


[ mean, XGBoost ]


Iteration number: 2 3s (2s) (23s), Current pipelines:  [[[ most_frequent, XGBoost ]]], BO objective: -0.9999999999999998


...

 |||| Now building the ensemble...

**Ensemble: **['[ iterative_k_neighbors, XGBoost ]', '[ median, XGBoost ]', '[ most_frequent, XGBoost ]']

**Ensemble weights: **[0.25229537 0.36119685 0.38650777]

**The ensemble did not help.**

[{'name': 'initial', 'aucprc': 0.01039209853612671},
 {'aucprc': 0.006940416282112655,
  'aucroc': 0.5962493036614914,
  'name': '[ median, XGBoost ]',
  'cv': 3,
  'iter': 0,
  'component_idx': 0,
  'hyperparameter_properties': [{'name': 'median'},
   {'name': 'XGBoost',
    'hyperparameters': {'model': "XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,\n              colsample_bynode=None, colsample_bytree=0.7880654774490066,\n              gamma=4.713279704681442, gpu_id=None, importance_type='gain',\n              interaction_constraints=None, learning_rate=0.4480875017386628,\n              max_delta_step=None, max_depth=7,\n              min_child_weight=1.7594675241891482, missing=nan,\n              monotone_constraints=None, n_estimators=50, n_jobs=None,\n              num_parallel_tree=None, random_state=None, reg_alpha=None,\n              reg_lambda=None, scale_pos_weight=None,\n              subsample=0.9711233497616982, tree_method=None,\n              

## Computing model predictions

In [21]:
AP_mdl.predict(X_)

(array([[1.       , 0.       ],
        [0.9966131, 0.0033869],
        [1.       , 0.       ],
        ...,
        [1.       , 0.       ],
        [1.       , 0.       ],
        [1.       , 0.       ]]),
 array([[1.       , 0.       ],
        [0.9966131, 0.0033869],
        [1.       , 0.       ],
        ...,
        [1.       , 0.       ],
        [1.       , 0.       ],
        [1.       , 0.       ]]))

## Compute performance via multi-fold cross-validation

In [23]:
model.evaluate_ens(X_, Y_, AP_mdl, n_folds=3, visualize=True, X_val_indexes=[])

[ Random Forest ]
[ Gradient Boosting ]
[ XGBoost ]


[ Random Forest ]
[ Gradient Boosting ]
[ XGBoost ]


Iteration number: 1 16s (16s) (47s), Current pipelines:  [[[ Random Forest ]]], [[[ Gradient Boosting ]]], [[[ XGBoost ]]], BO objective: 0.0


[ Random Forest ]
[ Gradient Boosting ]
[ XGBoost ]


Iteration number: 2 21s (10s) (31s), Current pipelines:  [[[ Random Forest ]]], [[[ Gradient Boosting ]]], [[[ XGBoost ]]], BO objective: -1.0000000000000004


...

---------------------------------------------------------


**Final Cross-validation score: **(0.010754419549904169, 0.0045629979398715305)

**Final Cross-validation score with ensembles: **(0.011552975276846435, 0.0032098761530330896)

---------------------------------------------------------


((0.010754419549904169, 0.0045629979398715305),
 (0.011552975276846435, 0.0032098761530330896),
 {'clf': {'roc_lst': [0.5675601147143171,
    0.5364135230531657,
    0.6484281709153211,
    0.6631016042780747,
    0.7185983009708738],
   'prc_lst': [0.006852069046771825,
    0.005940358377233659,
    0.020551837507961877,
    0.009704311701573074,
    0.010723521115980402],
   'roc_cur': 0.7185983009708738,
   'prc_cur': 0.010723521115980402},
  'clf_ens': {'roc_lst': [0.6018158504301787,
    0.6324936576218839,
    0.6306006957063496,
    0.7049543118218161,
    0.686658759929391],
   'prc_lst': [0.015280453880859834,
    0.006913172688631486,
    0.01641602190575411,
    0.00987399054978173,
    0.009281237359205016],
   'roc_cur': 0.686658759929391,
   'prc_cur': 0.009281237359205016}},
 [[{'name': 'initial', 'aucprc': 0.00980245552290891},
   {'aucprc': 0.006768763091619896,
    'aucroc': 0.544009096425115,
    'name': '[ Random Forest ]',
    'cv': 3,
    'iter': 0,
    'component

## Visualize data...

In [26]:
AP_mdl.visualize_data(X_)

## Visualize the model...

In [27]:
AP_mdl.APReport()

***Ensemble Report***

**----------------------**

**Rank0:   [ iterative_k_neighbors, XGBoost ],   Ensemble weight: 0.2522953737697456**

**----------------------**

{'model_list': [<models.imputers.iterative_k_neighbors object at 0x7fbf64232d50>, <models.classifiers.XGboost object at 0x7fbf6320c610>], 'explained': '[ , *GBoost is an open-source software library which provides the gradient boosting framework for C++, Java, Python, R, and Julia.* ]', 'image_name': None, 'classes': None, 'num_stages': 2, 'pipeline_stages': ['imputer', 'classifier'], 'name': '[ iterative_k_neighbors, XGBoost ]', 'analysis_mode': None, 'analysis_type': None}


...

***Kernel Report***

**Component 0**

**Members: ['XGBoost']**

  Mat52.       |                 value  |  constraints  |  priors
  variance     |     0.765128065717315  |      +ve      |        
  lengthscale  |  0.021327189746694973  |      +ve      |        


{'best_score_single_pipeline': 0.02165049492204985,
 'model_names_single_pipeline': '[ iterative_decision_tree, XGBoost ]',
 'ensemble_score': 0.01664557078126209,
 'ensemble_pipelines': ['[ iterative_k_neighbors, XGBoost ]',
  '[ median, XGBoost ]',
  '[ most_frequent, XGBoost ]'],
 'ensemble_pipelines_weight': [0.2522953737697456,
  0.3611968514171138,
  0.38650777481314064],
 'optimisation_metric': 'aucprc',
 'hyperparameter_properties': [{'name': 'iterative_decision_tree'},
  {'name': 'XGBoost',
   'hyperparameters': {'model': "XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,\n              colsample_bynode=1, colsample_bytree=0.8791341468540339,\n              gamma=0.6713188281096989, gpu_id=-1, importance_type='gain',\n              interaction_constraints='', learning_rate=0.46041937776928454,\n              max_delta_step=0, max_depth=3,\n              min_child_weight=1.6839447424385967, missing=nan,\n              monotone_constraints='()', n_estimators=1